In [1]:
import pandas as pd
import numpy as np
import json
import os
import random
import string
import re
import matplotlib.pyplot as plt
import pickle

from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error
import optuna.integration.lightgbm as lgbo
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import optuna
import umap
pd.set_option('display.max_columns', 2000)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

SEED = 777



In [2]:
df_train = pd.read_pickle('../data/train/full_comp.pickle')
df_train = df_train[df_train['cites'].isnull() == False].reset_index(drop=True)
df_train['cites'] = np.log1p(df_train['cites'], dtype=np.float64)
df_train.shape


(15117, 2002)

In [3]:
dorop_cols = []
for col in df_train.columns:
    if '_x' in col:
        dorop_cols.append(col)
len(dorop_cols), dorop_cols
df_train = df_train.drop(dorop_cols, axis=1)


In [4]:
NFOLDS = 
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)

df_train['fold_no'] = [0 for x in range(len(df_train))]
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold_no'] = fold
df_train.head()


0.090434         0.001255        -0.047360        -0.103646   

   roberta_vec_587  roberta_vec_588  roberta_vec_589  roberta_vec_590  \
0         0.041590        12.282816         0.079741        -0.060993   
1         0.033566        12.149046         0.052788        -0.019711   
2        -0.036725        11.943683         0.009125        -0.021048   
3         0.084762        11.718389        -0.038374        -0.011906   
4         0.108661        11.651844        -0.063221        -0.013029   

   roberta_vec_591  roberta_vec_592  roberta_vec_593  roberta_vec_594  \
0         0.063440        -0.268049        -0.094870         0.024665   
1         0.077738        -0.249763        -0.025224         0.030602   
2         0.045698        -0.227533        -0.049069        -0.027898   
3        -0.001632        -0.224642        -0.079878        -0.099328   
4         0.032770        -0.256653         0.004424        -0.158540   

   roberta_vec_595  roberta_vec_596  roberta_vec_597  roberta_vec_598  \
0         0.001880         0.187180         0.040667         0.006068   
1        -0.058953         0.241092         0.107833        -0.050415   
2         0.021803         0.141909         0.143262         0.028343   
3         0.124719         0.179040         0.142606        -0.036361   
4         0.057062         0.279952         0.185350        -0.026612   

   roberta_vec_599  roberta_vec_600  roberta_vec_601  roberta_vec_602  \
0         0.023697         0.187934        -0.221596         0.068942   
1        -0.096370         0.116405        -0.205613         0.103472   
2         0.013603         0.211610        -0.150498        -0.003177   
3        -0.080203         0.120659        -0.049413         0.000616   
4        -0.073181         0.144206        -0.092292         0.073268   

   roberta_vec_603  roberta_vec_604  roberta_vec_605  roberta_vec_606  \
0         0.005736         0.098046         0.084538         0.091436   
1         0.020359         0.087780         0.165174         0.151956   
2         0.041957        -0.032630         0.211481         0.146104   
3         0.114762         0.136759         0.128589         0.083282   
4         0.087679         0.119163         0.097052         0.128711   

   roberta_vec_607  roberta_vec_608  roberta_vec_609  roberta_vec_610  \
0        -0.074916         0.082979         0.174427        -0.010615   
1        -0.155135         0.048719         0.136277         0.014289   
2        -0.185986         0.156548         0.081175         0.050995   
3        -0.119280         0.175465         0.175211         0.079239   
4        -0.163158         0.196666         0.096923         0.021693   

   roberta_vec_611  roberta_vec_612  roberta_vec_613  roberta_vec_614  \
0         0.956542        -0.157765         0.202592        -0.065107   
1         0.941260        -0.143702         0.072925        -0.135912   
2         0.899738        -0.161586         0.089926        -0.130796   
3         0.902776        -0.046504         0.255126        -0.002165   
4         0.768567        -0.057968         0.151456        -0.043386   

   roberta_vec_615  roberta_vec_616  roberta_vec_617  roberta_vec_618  \
0         0.086034         0.082509         0.116933        -0.130661   
1         0.126020         0.117271         0.201530        -0.107007   
2        -0.123190         0.086665         0.235653        -0.042723   
3        -0.024777         0.089408         0.198304         0.016981   
4        -0.004558         0.144182         0.187076         0.072016   

   roberta_vec_619  roberta_vec_620  roberta_vec_621  roberta_vec_622  \
0         0.056648        -0.001675        -0.010225         0.085153   
1         0.218389         0.065998        -0.083668         0.053277   
2         0.308070         0.091798        -0.021499        -0.014432   
3         0.230631         0.072909        -0.130362        -0.013177   
4         0.245870         0.112004        -0.171178       

In [5]:
importance_0_cols = ['econ_y', 'eess_y', 'nlin_y', 'physics_y', 'acc-phys_y',
       'adap-org_y', 'alg-geom_y', 'ao-sci_y', 'astro-ph_y', 'atom-ph_y',
       'bayes-an_y', 'chao-dyn_y', 'chem-ph_y', 'cmp-lg_y', 'comp-gas_y',
       'dg-ga_y', 'funct-an_y', 'gr-qc_y', 'math-ph_y', 'mtrl-th_y',
       'nucl-ex_y', 'patt-sol_y', 'plasm-ph_y', 'q-alg_y', 'q-fin_y',
       'solv-int_y', 'supr-con_y', 'acc_y', 'adap_y', 'alg_y', 'ao_y',
       'astro_y', 'atom_y', 'bayes_y', 'chao_y', 'chem_y', 'cmp_y',
       'comp_y', 'cond_y', 'dg_y', 'econ', 'eess', 'funct_y', 'gr_y',
       'math', 'mtrl_y', 'nlin', 'patt_y', 'physics', 'plasm_y',
       'quant_y', 'solv_y', 'stat', 'supr_y', 'astro-ph.ga',
       'astro-ph.he', 'astro-ph.sr', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.other', 'cond-mat.soft',
       'cond-mat.stat-mech', 'cs.ai', 'cs.ar', 'cs.cc', 'cs.ce', 'cs.cg',
       'cs.cl', 'cs.cr', 'cs.cv', 'cs.cy', 'cs.db', 'cs.dc', 'cs.dl',
       'cs.dm', 'cs.et', 'cs.fl', 'cs.gl', 'cs.gr', 'cs.gt', 'cs.hc',
       'cs.ir', 'cs.it', 'cs.lo', 'cs.ma', 'cs.mm', 'cs.ms', 'cs.na',
       'cs.ne', 'cs.ni', 'cs.oh', 'cs.os', 'cs.pf', 'cs.pl', 'cs.ro',
       'cs.sc', 'cs.sd', 'cs.se', 'cs.sy', 'econ.em', 'econ.gn',
       'econ.th', 'eess.as', 'eess.iv', 'eess.sp', 'eess.sy', 'math.ac',
       'math.ap', 'math.at', 'math.ca', 'math.ct', 'math.cv', 'math.dg',
       'math.ds', 'math.fa', 'math.gm', 'math.gn', 'math.gr', 'math.gt',
       'math.ho', 'math.it', 'math.kt', 'math.lo', 'math.mg', 'math.mp',
       'math.na', 'math.nt', 'math.oa', 'math.oc', 'math.qa', 'math.ra',
       'math.rt', 'math.sg', 'math.sp', 'math.st', 'nlin.ao', 'nlin.cd',
       'nlin.cg', 'nlin.ps', 'nlin.si', 'physics.acc-ph', 'physics.ao-ph',
       'physics.app-ph', 'physics.atm-clus', 'physics.bio-ph',
       'physics.chem-ph', 'physics.class-ph', 'physics.comp-ph',
       'physics.data-an', 'physics.ed-ph', 'physics.flu-dyn',
       'physics.gen-ph', 'physics.geo-ph', 'physics.hist-ph',
       'physics.ins-det', 'physics.med-ph', 'physics.optics',
       'physics.plasm-ph', 'physics.pop-ph', 'physics.soc-ph', 'q-bio.bm',
       'q-bio.cb', 'q-bio.gn', 'q-bio.mn', 'q-bio.nc', 'q-bio.ot',
       'q-bio.pe', 'q-bio.qm', 'q-bio.sc', 'q-bio.to', 'q-fin.cp',
       'q-fin.ec', 'q-fin.gn', 'q-fin.mf', 'q-fin.pm', 'q-fin.pr',
       'q-fin.rm', 'q-fin.st', 'q-fin.tr', 'stat.ap', 'stat.co',
       'stat.me', 'stat.ml', 'stat.ot', 'doi_cites_min_doi_id_label',
       'doi_cites_min_pub_publisher_label',
       'doi_cites_median_pub_publisher_label', 'doi_cites_min_update_ym',
       'doi_cites_min_first_created_ym', 'doi_cites_min_license_label',
       'doi_cites_max_license_label', 'doi_cites_q10_license_label',
       'doi_cites_q75_license_label', 'doi_cites_min_category_main_label',
       'doi_cites_q10_category_main_label',
       'doi_cites_q25_category_main_label',
       'doi_cites_min_category_main_detail_label',
       'doi_cites_median_category_main_detail_label',
       'doi_cites_q10_category_main_detail_label',
       'doi_cites_q25_category_main_detail_label',
       'doi_cites_min_category_name_parent_label',
       'doi_cites_q10_category_name_parent_label',
       'doi_cites_min_category_name_parent_main_label',
       'doi_cites_q10_category_name_parent_main_label',
       'doi_cites_min_category_name_label',
       'pred_doi_cites_min_doi_id_label',
       'pred_doi_cites_min_pub_publisher_label',
       'pred_doi_cites_median_pub_publisher_label',
       'pred_doi_cites_q75_pub_publisher_label',
       'pred_doi_cites_min_update_ym', 'pred_doi_cites_q10_update_ym',
       'pred_doi_cites_min_first_created_ym',
       'pred_doi_cites_q10_first_created_ym',
       'pred_doi_cites_mean_license_label',
       'pred_doi_cites_count_license_label',
       'pred_doi_cites_sum_license_label',
       'pred_doi_cites_min_license_label',
       'pred_doi_cites_std_license_label',
       'pred_doi_cites_q10_license_label',
       'pred_doi_cites_q25_license_label',
       'pred_doi_cites_q75_license_label',
       'pred_doi_cites_mean_category_main_label',
       'pred_doi_cites_min_category_main_label',
       'pred_doi_cites_median_category_main_label',
       'pred_doi_cites_q10_category_main_label',
       'pred_doi_cites_q25_category_main_label',
       'pred_doi_cites_mean_category_main_detail_label',
       'pred_doi_cites_sum_category_main_detail_label',
       'pred_doi_cites_min_category_main_detail_label',
       'pred_doi_cites_median_category_main_detail_label',
       'pred_doi_cites_q10_category_main_detail_label',
       'pred_doi_cites_q25_category_main_detail_label',
       'pred_doi_cites_q75_category_main_detail_label',
       'pred_doi_cites_min_category_name_parent_label',
       'pred_doi_cites_median_category_name_parent_label',
       'pred_doi_cites_q10_category_name_parent_label',
       'pred_doi_cites_q25_category_name_parent_label',
       'pred_doi_cites_min_category_name_parent_main_label',
       'pred_doi_cites_median_category_name_parent_main_label',
       'pred_doi_cites_q10_category_name_parent_main_label',
       'pred_doi_cites_q25_category_name_parent_main_label',
       'pred_doi_cites_min_category_name_label',
       'diff_rate_doi_cites_pred_doi_cites',
       'diff_rate_doi_cites_mean_submitter_label_pred_doi_cites_mean_submitter_label',
       'diff_rate_doi_cites_mean_doi_id_label_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_doi_id_label_pred_doi_cites_mean_doi_id_label',
       'diff_rate_doi_cites_mean_doi_id_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_author_first_label_pred_doi_cites_mean_author_first_label',
       'diff_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_doi_cites_mean_pub_publisher_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_update_ym_pred_doi_cites_mean_update_ym',
       'diff_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_update_ym_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_doi_cites_mean_first_created_ym_pred_doi_cites_mean_first_created_ym',
       'diff_rate_doi_cites_mean_license_label_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_license_label_pred_doi_cites_mean_license_label',
       'diff_rate_doi_cites_mean_license_label_pred_doi_cites_mean_license_label',
       'diff_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_doi_cites_mean_category_main_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_doi_cites_mean_category_name_parent_label',
       'diff_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_doi_id_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_update_ym',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_license_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_update_ym',
       'diff_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_license_label',
       'diff_rate_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_license_label',
       'diff_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_doi_cites_mean_category_name_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_doi_cites_mean_category_name_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_label_pred_doi_cites_mean_category_name_label',
       'diff_pred_doi_cites_mean_submitter_label_pred_doi_cites_mean_doi_id_label',
       'diff_rate_pred_doi_cites_mean_submitter_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_author_first_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_update_ym',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_first_created_ym',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_license_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_name_label',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_rate_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_update_ym',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_update_ym',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_first_created_ym',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_first_created_ym',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_license_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_license_label',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_license_label',
       'diff_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_name_parent_label',
       'diff_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_first_created_ym_pred_doi_cites_mean_license_label',
       'diff_rate_pred_doi_cites_mean_first_created_ym_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_name_label',
       'diff_pred_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_name_label',
       'is_null_comments', 'is_null_journal-ref']


In [6]:
importance_0_cols.extend(['id', 'authors', 'title', 'comments',
    'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique',
    'submitter_label','doi_id_label','author_first_label','pub_publisher_label',
    'license_label','category_main_label','category_name_parent_label','category_name_parent_main_label', 'category_name_label',
    #'diff_pred_doi_cites', 'rate_pred_doi_cites'
    ])

In [10]:
# パラメーター調整
result_y = []
lgb_result_proba_test = []
lgb_result_proba_valid = []

fold_no = 2
test_fold_no = fold_no
valid_fold_no = fold_no + 1
if valid_fold_no == NFOLDS:
    valid_fold_no = 0

# train
train = df_train.copy()
y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
y_valid = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values

train = train.drop(importance_0_cols, axis=1)
x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
x_valid = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]

'''
# target encoding
target = 'doi_cites'
key = 'pred_doi_cites'
x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

keys = ['update_ym']
target = 'cites'
for key in keys:
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
'''

# drop
x_train = x_train.drop(['cites', 'fold_no'], axis=1)
x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'num_iterations': 10000,
    'early_stopping_rounds': 50
}
best_params, history = {}, []
trains = lgb.Dataset(x_train, y_train)
valids = lgb.Dataset(x_valid, y_valid)

best = lgbo.train(params, trains, valid_sets=valids,
                    verbose_eval=False,
                    num_boost_round=100,
                    early_stopping_rounds=50)
best.params

    | 2/20 [00:29<04:38, 15.47s/it][I 2021-03-28 04:18:16,261] Trial 41 finished with value: 0.5115401856119461 and parameters: {'lambda_l1': 5.862010160751429, 'lambda_l2': 0.00019899079419315966}. Best is trial 41 with value: 0.5115401856119461.

regularization_factors, val_score: 0.510270:  10%|#         | 2/20 [00:29<04:38, 15.47s/it][LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.065566 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 355276
[LightGBM] [Info] Number of data points in the train set: 12093, number of used features: 1555
[LightGBM] [Info] Start training from score 2.576288

regularization_factors, val_score: 0.510270:  15%|#5        | 3/20 [00:44<04:19, 15.29s/it][I 2021-03-28 04:18:31,131] Trial 42 finished with value: 0.5117071008501705 and parameters: {'lambda_l1': 1.0281343117286185e-07, 'lambda_l2': 0.07362007087857975}. Best is trial 41 with value: 0.5115401856119461.

reg

{'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.01,
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 29,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 10,
 'num_iterations': 10000,
 'early_stopping_round': 50}

In [7]:
result_y_test = []
result_y_valid = []
lgb_result_proba_test = []
lgb_result_proba_valid = []

'''
lgb_params = {'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.01,
 'feature_pre_filter': False,
 'lambda_l1': 8.856362720452578e-07,
 'lambda_l2': 0.0002814044321572516,
 'num_leaves': 31,
 'feature_fraction': 0.42,
 'bagging_fraction': 0.8762325382386713,
 'bagging_freq': 4,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 50,}
'''

lgb_params = {'objective': 'regression',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'num_leaves': 31,
 'feature_fraction': 0.42,
 'bagging_fraction': 0.8762325382386713,
 'bagging_freq': 4,
 'learning_rate': 0.01,
 'lambda_l1': 8.856362720452578e-07,
 'lambda_l2': 0.0002814044321572516,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 50}

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([valid_fold_no])]['cites'].values # test_fold_no, 
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(importance_0_cols, axis=1)

    x_train = train[~train['fold_no'].isin([valid_fold_no])] # test_fold_no, 
    x_valid = train[train['fold_no'] == valid_fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    '''
    # target encoding
    target = 'doi_cites'
    key = 'pred_doi_cites'
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

    keys = ['update_ym']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
    '''

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    #x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                eval_metric='rmse',
                verbose=100,
                early_stopping_rounds=50,
    )

    pickle.dump(model, open(f'../models/lgb_{fold_no}.pickle', 'wb'))

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    #lgb_result_proba_test.extend(fold_result_test)
    lgb_result_proba_valid.extend(fold_result_valid)
    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    #print(f"fold {fold_no} lgb score: {rmsle}")


print("valid", mean_squared_error(result_y_valid, lgb_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, lgb_result_proba_test, squared=False))



/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Warning] lambda_l1 is set=8.856362720452578e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.856362720452578e-07
[LightGBM] [Warning] bagging_fraction is set=0.8762325382386713, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8762325382386713
[LightGBM] [Warning] lambda_l2 is set=0.0002814044321572516, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0002814044321572516
[LightGBM] [Warning] feature_fraction is set=0.42, colsample_bytree=1.0 will be ignored. Current value: feature_

In [22]:
fold_no = 2
test_fold_no = fold_no
valid_fold_no = fold_no + 1
if valid_fold_no == NFOLDS:
    valid_fold_no = 0

# train
train = df_train.copy()
y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
y_test = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values

train = train.drop(importance_0_cols, axis=1)
train = train.fillna(0)

x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
x_test = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]

x_train = x_train.drop(['cites', 'fold_no'], axis=1)
x_test = x_test.drop(['cites', 'fold_no'], axis=1)

def objective(trial):
    bootstrap = trial.suggest_categorical('bootstrap',[True,False])
    max_depth = trial.suggest_int('max_depth', 1, 28)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 60)
    n_estimators =  trial.suggest_int('n_estimators', 1, 300)
    min_samples_split = trial.suggest_int('min_samples_split',2, 100)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1, 20)

    model = RandomForestRegressor(bootstrap = bootstrap, criterion = 'mse',
                                 max_depth = max_depth, 
                                 max_leaf_nodes = max_leaf_nodes,
                                 n_estimators = n_estimators,
                                 min_samples_split = min_samples_split,min_samples_leaf = min_samples_leaf,
                                 n_jobs=-1)

    model.fit(x_train, y_train)
    return mean_squared_error(y_test, model.predict(x_test), squared=False)


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-03-28 05:02:29,175] A new study created in memory with name: no-name-1b4f71d6-c821-4936-8e06-9043ee8cc1d3
[I 2021-03-28 05:02:37,927] Trial 0 finished with value: 0.5591247978476644 and parameters: {'bootstrap': True, 'max_depth': 10, 'max_leaf_nodes': 13, 'n_estimators': 47, 'min_samples_split': 11, 'min_samples_leaf': 14}. Best is trial 0 with value: 0.5591247978476644.
[I 2021-03-28 05:03:34,668] Trial 1 finished with value: 0.5419898130448385 and parameters: {'bootstrap': True, 'max_depth': 14, 'max_leaf_nodes': 45, 'n_estimators': 220, 'min_samples_split': 20, 'min_samples_leaf': 11}. Best is trial 1 with value: 0.5419898130448385.
[I 2021-03-28 05:04:33,260] Trial 2 finished with value: 0.5791650456165397 and parameters: {'bootstrap': False, 'max_depth': 26, 'max_leaf_nodes': 58, 'n_estimators': 134, 'min_samples_split': 92, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.5419898130448385.
[I 2021-03-28 05:04:59,086] Trial 3 finished with value: 0.6003334896452377 

In [8]:
result_y_test = []
result_y_valid = []
#rf_result_proba_test = []
rf_result_proba_valid = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    train = train.fillna(0)

    y_train = train[~train['fold_no'].isin([valid_fold_no])]['cites'].values # test_fold_no, 
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(importance_0_cols, axis=1)

    x_train = train[~train['fold_no'].isin([valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    model = RandomForestRegressor(
        max_depth=23,
        max_leaf_nodes=59,
        n_estimators=34,
        min_samples_split=40,
        min_samples_leaf=14,
        criterion='mse',
        random_state=SEED,
        n_jobs=-1)
    model.fit(x_train, y_train)
    pickle.dump(model, open(f'../models/rf_{fold_no}.pickle', 'wb'))

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    #rf_result_proba_test.extend(fold_result_test)
    rf_result_proba_valid.extend(fold_result_valid)
    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print('done...', fold_no)

print("valid", mean_squared_error(result_y_valid, rf_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, rf_result_proba_test, squared=False))



valid 0.5236471937748199


In [74]:
def objective(trial):
    categorical_features_indices = np.where(x_train.dtypes == 'category')[0]

    train_pool = Pool(x_train, y_train, cat_features=categorical_features_indices)
    test_pool = Pool(x_test, y_test, cat_features=categorical_features_indices)
    
    # Parameters
    param = {
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "iterations": 1000,
        "learning_rate": 0.03,
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "random_seed": SEED,
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    model = CatBoostRegressor(**param)
    model.fit(train_pool, eval_set=test_pool, verbose=200, early_stopping_rounds=20)
    
    preds = model.predict(test_pool)
    return mean_squared_error(y_test, preds, squared=False)

In [75]:
fold_no = 2
test_fold_no = fold_no
valid_fold_no = fold_no + 1
if valid_fold_no == NFOLDS:
    valid_fold_no = 0

# train
train = df_train.copy()
y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
y_test = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values

train = train.drop(importance_0_cols, axis=1)
train['diff_pred_doi_cites'] = train['pred_doi_cites'] - train['doi_cites']
train['rate_pred_doi_cites'] = train['pred_doi_cites'] / (train['doi_cites'] + 1)

x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
x_test = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]

x_train = x_train.drop(['cites', 'fold_no'], axis=1)
x_test = x_test.drop(['cites', 'fold_no'], axis=1)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, timeout=600)


[I 2021-03-27 13:14:24,743] A new study created in memory with name: no-name-3c4a5d94-a509-479b-bb32-0534db732653
0:	learn: 1.1743888	test: 1.1671023	best: 1.1671023 (0)	total: 8.96ms	remaining: 8.95s
200:	learn: 0.5220326	test: 0.5447488	best: 0.5447488 (200)	total: 1.28s	remaining: 5.08s
400:	learn: 0.5033216	test: 0.5310702	best: 0.5310702 (400)	total: 2.41s	remaining: 3.6s
600:	learn: 0.4937408	test: 0.5264670	best: 0.5264670 (600)	total: 3.5s	remaining: 2.33s
800:	learn: 0.4871515	test: 0.5240054	best: 0.5240054 (800)	total: 4.64s	remaining: 1.15s
[I 2021-03-27 13:14:31,065] Trial 0 finished with value: 0.5227816675067647 and parameters: {'colsample_bylevel': 0.09178193753637694, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.22889708929840308}. Best is trial 0 with value: 0.5227816675067647.
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5227816692
bestIteration = 950

Shrink model to first 951 iterations.
0:	learn: 1.1701

In [76]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 15
Best trial:
  Value: 0.5125028838315051
  Params: 
    colsample_bylevel: 0.052922169991835896
    depth: 6
    boosting_type: Ordered
    bootstrap_type: MVS


In [9]:
from catboost import Pool
SEED = 777
result_y_test = []
result_y_valid = []
cat_result_proba_test = []
cat_result_proba_valid = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(importance_0_cols, axis=1)

    x_train = train[~train['fold_no'].isin([valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    #x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    #cat_features = np.where(x_train.dtypes == 'category')[0]
    train_pool = Pool(x_train, y_train)
    validate_pool = Pool(x_valid, y_valid)
    #test_pool = Pool(x_test, y_test, cat_features=cat_features)
    
    model = CatBoostRegressor(
        loss_function='RMSE',
        eval_metric="RMSE",
        colsample_bylevel=0.052922169991835896,
        depth=6,
        boosting_type='Ordered',
        bootstrap_type='MVS',
        iterations=10000,
        random_seed=SEED,
    )
    #model = lgb.LGBMRegressor(**lgb_params)
    model.fit(train_pool,
                eval_set=validate_pool,
                verbose=100,
                early_stopping_rounds=20,
    )

    pickle.dump(model, open(f'../models/cat_{fold_no}.pickle', 'wb'))

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    #cat_result_proba_test.extend(fold_result_test)
    cat_result_proba_valid.extend(fold_result_valid)

    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    #print(f"fold {fold_no} lgb score: {rmsle}")

print("valid", mean_squared_error(result_y_valid, cat_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, cat_result_proba_test, squared=False))


Learning rate set to 0.017979
0:	learn: 1.1757940	test: 1.1736326	best: 1.1736326 (0)	total: 88.7ms	remaining: 14m 47s
100:	learn: 0.5777559	test: 0.5750382	best: 0.5750382 (100)	total: 5.63s	remaining: 9m 11s
200:	learn: 0.5216260	test: 0.5130720	best: 0.5130720 (200)	total: 11.2s	remaining: 9m 3s
300:	learn: 0.5070068	test: 0.4993360	best: 0.4993360 (300)	total: 16.7s	remaining: 8m 58s
400:	learn: 0.4984863	test: 0.4934983	best: 0.4934983 (400)	total: 22.3s	remaining: 8m 54s
500:	learn: 0.4917994	test: 0.4894988	best: 0.4894988 (500)	total: 27.9s	remaining: 8m 48s
600:	learn: 0.4857147	test: 0.4868635	best: 0.4868635 (600)	total: 33.5s	remaining: 8m 43s
700:	learn: 0.4789177	test: 0.4850007	best: 0.4849902 (698)	total: 39.1s	remaining: 8m 39s
800:	learn: 0.4727986	test: 0.4835293	best: 0.4834929 (796)	total: 44.8s	remaining: 8m 34s
900:	learn: 0.4668707	test: 0.4827211	best: 0.4827211 (900)	total: 50.4s	remaining: 8m 28s
1000:	learn: 0.4612808	test: 0.4819116	best: 0.4819116 (1000)	t

In [12]:
result_y_valid = []
reg_result_proba_valid = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(importance_0_cols, axis=1)
    train = train.fillna(0)

    x_train = train[~train['fold_no'].isin([valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    #x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_valid = scaler.fit_transform(x_valid)

    model = linear_model.Ridge()
    model.fit(x_train, y_train)
    pickle.dump(model, open(f'../models/reg_{fold_no}.pickle', 'wb'))

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    #cat_result_proba_test.extend(fold_result_test)
    reg_result_proba_valid.extend(fold_result_valid)

    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"done... fold {fold_no}")


print("valid", mean_squared_error(result_y_valid, reg_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, cat_result_proba_test, squared=False))


done... fold 0
done... fold 1
done... fold 2
done... fold 3
done... fold 4
done... fold 5
done... fold 6
done... fold 7
done... fold 8
done... fold 9
valid 0.5191902200700491


In [81]:
def build_model():
  input = tf.keras.layers.Input(x_train.shape[1], name="input")
  x = tf.keras.layers.Dense(256, activation='relu')(input)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.1)(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, name='outputs')(x)
  optimizer = tf.keras.optimizers.Adam()

  model = tf.keras.Model(inputs=input, outputs=outputs)
  model.compile(
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
    optimizer=optimizer)
  return model
  
model = build_model()
model.summary()

Model: "functional_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1555)]            0         
_________________________________________________________________
dense_74 (Dense)             (None, 256)               398336    
_________________________________________________________________
dropout_24 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_25 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_76 (Dense)             (None, 128)               16512     
_________________________________________________________________
outputs (Dense)              (None, 1)               

In [82]:
NFOLDS  = 10

In [83]:
result_y_valid = []
nn_result_proba_valid = []
for fold_no in range(NFOLDS):
    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'] != fold_no)]['cites'].values
    y_valid = train[train['fold_no'] == fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(importance_0_cols, axis=1)
    train = train.fillna(0)

    x_train = train[~train['fold_no'] != fold_no]
    x_valid = train[train['fold_no'] == fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    #x_test = x_test.drop(['cites', 'fold_no'], axis=1)
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_valid = scaler.fit_transform(x_valid)

    model = build_model()
    checkpoint_filepath = f'./models/nn_{fold_no}_weights.hdf5'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        save_best_only=True
    )
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    history = model.fit(
         x=x_train,
         y=y_train,
         validation_data=(x_valid, y_valid),
         epochs=100,
         verbose=1,
         callbacks=[early_stop, model_checkpoint_callback])

    model = model.load_weights(checkpoint_filepath)
    result_y_valid.extend(y_valid)
    #cat_result_proba_test.extend(fold_result_test)
    nn_result_proba_valid.extend(model.predict(x_valid))

    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"done... fold {fold_no}")

print("valid", mean_squared_error(result_y_valid, nn_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, cat_result_proba_test, squared=False))

Epoch 1/100
378/378 [==============================] - 1s 2ms/step - loss: 0.8304 - root_mean_squared_error: 0.9113 - val_loss: 0.4810 - val_root_mean_squared_error: 0.6935
Epoch 2/100
378/378 [==============================] - 1s 2ms/step - loss: 0.4936 - root_mean_squared_error: 0.7026 - val_loss: 0.3901 - val_root_mean_squared_error: 0.6246
Epoch 3/100
378/378 [==============================] - 1s 2ms/step - loss: 0.3987 - root_mean_squared_error: 0.6314 - val_loss: 0.5783 - val_root_mean_squared_error: 0.7605
Epoch 4/100
378/378 [==============================] - 1s 2ms/step - loss: 0.3746 - root_mean_squared_error: 0.6120 - val_loss: 0.5138 - val_root_mean_squared_error: 0.7168
Epoch 5/100
378/378 [==============================] - 1s 2ms/step - loss: 0.3532 - root_mean_squared_error: 0.5943 - val_loss: 0.4326 - val_root_mean_squared_error: 0.6577
Epoch 6/100
378/378 [==============================] - 1s 2ms/step - loss: 0.3308 - root_mean_squared_error: 0.5752 - val_loss: 0.2851 

KeyboardInterrupt: 

In [29]:
valid_fold_no

1

In [14]:
ids = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    ids.extend(train[train['fold_no'] == valid_fold_no]['id'].values)

    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values
len(ids)

15117

In [54]:
df_result['y'][:10]

0    2.197225
1    0.693147
2    3.044522
3    3.178054
4    2.833213
5    0.693147
6    3.465736
7     3.89182
8    1.098612
9    0.693147
Name: y, dtype: object

In [15]:
df_result = pd.DataFrame(ids, columns=['id'])
df_result = pd.concat([df_result, pd.DataFrame(lgb_result_proba_valid, columns=['lgb'])], axis=1)
df_result = pd.concat([df_result, pd.DataFrame(cat_result_proba_valid, columns=['cat'])], axis=1)
df_result = pd.concat([df_result, pd.DataFrame(rf_result_proba_valid, columns=['rf'])], axis=1)
df_result = pd.concat([df_result, pd.DataFrame(reg_result_proba_valid, columns=['reg'])], axis=1)
df_result = pd.concat([df_result, pd.DataFrame(result_y_valid, columns=['y'])], axis=1)
df_result.head(3)

,id,lgb,cat,rf,reg,y
0,1601.03743,1.904554,1.944293,2.110282,1.965835,2.197225
1,1903.05430,1.276475,1.190545,1.212217,1.248960,0.693147
2,hep-lat/0602030,2.836751,2.757892,2.955130,2.896053,3.044522


In [16]:
df_result.to_pickle('./first_layer_proba.pickle')

In [2]:
df_result = pd.read_pickle('./first_layer_proba.pickle')

In [17]:
df_result.shape

(15117, 6)

In [18]:
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)
df_result['fold_no'] = [0 for x in range(len(df_result))]
for fold, (train_idx, val_idx) in enumerate(kf.split(df_result)):
    df_result.loc[val_idx, 'fold_no'] = fold
df_result.head()

,id,lgb,cat,rf,reg,y,fold_no
0,1601.03743,1.904554,1.944293,2.110282,1.965835,2.197225,2
1,1903.05430,1.276475,1.190545,1.212217,1.248960,0.693147,3
2,hep-lat/0602030,2.836751,2.757892,2.955130,2.896053,3.044522,1
3,0908.1669,3.189939,3.265711,3.344978,3.268603,3.178054,1
4,0802.4265,2.859747,2.786780,2.684017,2.530880,2.833213,1


In [19]:
df_result.groupby('fold_no').count()

,id,lgb,cat,rf,reg,y
fold_no,,,,,,
0,3024,3024,3024,3024,3024,3024
1,3024,3024,3024,3024,3024,3024
2,3023,3023,3023,3023,3023,3023
3,3023,3023,3023,3023,3023,3023
4,3023,3023,3023,3023,3023,3023


In [20]:

result_pred = []
result_y = []
for fold_no in range(5):
    X = df_result.copy()
    x_train = X[X['fold_no'] != fold_no]
    x_valid = X[X['fold_no'] == fold_no]
    y_train = x_train['y']
    y_valid = x_valid['y']
    x_train = x_train.drop(['id', 'fold_no', 'y'], axis=1)
    x_valid = x_valid.drop(['id', 'fold_no', 'y'], axis=1)


    model = SVR(kernel="linear", C=0.72, epsilon=0.465)
    model.fit(x_train, y_train)
    pred = model.predict(x_valid)
    result_y.extend(y_valid)
    result_pred.extend(pred)
    pickle.dump(model, open(f'../models/2nd_svr_{fold_no}.pickle', 'wb'))

mean_squared_error(result_y, result_pred, squared=False)

0.49331739186014323

In [21]:

result_pred_r = []
result_y = []
for fold_no in range(5):
    X = df_result.copy()
    x_train = X[X['fold_no'] != fold_no]
    x_valid = X[X['fold_no'] == fold_no]
    y_train = x_train['y']
    y_valid = x_valid['y']
    x_train = x_train.drop(['id', 'fold_no', 'y'], axis=1)
    x_valid = x_valid.drop(['id', 'fold_no', 'y'], axis=1)

    model = linear_model.Ridge()
    model.fit(x_train, y_train)
    pred = model.predict(x_valid)
    result_y.extend(y_valid)
    result_pred_r.extend(pred)
    pickle.dump(model, open(f'../models/2nd_ridge_{fold_no}.pickle', 'wb'))

mean_squared_error(result_y, result_pred_r, squared=False)


0.4932670372385315

In [301]:
a = (np.array(result_pred)  + np.array(result_pred_r)) /2
mean_squared_error(result_y, a, squared=False)

0.4932649098344016

In [298]:
pd.concat([pd.DataFrame(result_pred), pd.DataFrame(result_pred_r)], axis=1)

,0,0
0,1.735784,1.746114
1,2.973110,2.985532
2,3.725636,3.723628
3,1.986350,1.995129
4,1.795368,1.806691
...,...,...
15112,4.614348,4.599155
15113,1.730040,1.726163
15114,3.989737,3.995231
15115,2.168759,2.172269


0.4932649098344016

0.4934494796935973

In [84]:
df_train.head(1)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,pred_doi_cites,category_main_detail,category_main,cs_y,econ_y,eess_y,math_y,nlin_y,physics_y,stat_y,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_label,doi_cites_q25_doi_id_label,doi_cites_q75_doi_id_label,doi_cites_mean_pub_publisher_label,doi_cites_count_pub_publisher_label,doi_cites_sum_pub_publisher_label,doi_cites_min_pub_publisher_label,doi_cites_max_pub_publisher_label,doi_cites_median_pub_publisher_label,doi_cites_std_pub_publisher_label,doi_cites_q10_pub_publisher_label,doi_cites_q25_pub_publisher_label,doi_cites_q75_pub_publisher_label,doi_cites_mean_submitter_label,doi_cites_count_submitter_label,doi_cites_sum_submitter_label,doi_cites_min_submit

In [85]:
df_result.head(1)

,id,lgb,cat,rf,reg,y
0,1601.03743,1.904554,1.944293,2.110282,1.965835,2.197225


In [90]:
importance_0_cols2 = ['econ_y', 'eess_y', 'nlin_y', 'physics_y', 'acc-phys_y',
       'adap-org_y', 'alg-geom_y', 'ao-sci_y', 'astro-ph_y', 'atom-ph_y',
       'bayes-an_y', 'chao-dyn_y', 'chem-ph_y', 'cmp-lg_y', 'comp-gas_y',
       'dg-ga_y', 'funct-an_y', 'gr-qc_y', 'math-ph_y', 'mtrl-th_y',
       'nucl-ex_y', 'patt-sol_y', 'plasm-ph_y', 'q-alg_y', 'q-fin_y',
       'solv-int_y', 'supr-con_y', 'acc_y', 'adap_y', 'alg_y', 'ao_y',
       'astro_y', 'atom_y', 'bayes_y', 'chao_y', 'chem_y', 'cmp_y',
       'comp_y', 'cond_y', 'dg_y', 'econ', 'eess', 'funct_y', 'gr_y',
       'math', 'mtrl_y', 'nlin', 'patt_y', 'physics', 'plasm_y',
       'quant_y', 'solv_y', 'stat', 'supr_y', 'astro-ph.ga',
       'astro-ph.he', 'astro-ph.sr', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.other', 'cond-mat.soft',
       'cond-mat.stat-mech', 'cs.ai', 'cs.ar', 'cs.cc', 'cs.ce', 'cs.cg',
       'cs.cl', 'cs.cr', 'cs.cv', 'cs.cy', 'cs.db', 'cs.dc', 'cs.dl',
       'cs.dm', 'cs.et', 'cs.fl', 'cs.gl', 'cs.gr', 'cs.gt', 'cs.hc',
       'cs.ir', 'cs.it', 'cs.lo', 'cs.ma', 'cs.mm', 'cs.ms', 'cs.na',
       'cs.ne', 'cs.ni', 'cs.oh', 'cs.os', 'cs.pf', 'cs.pl', 'cs.ro',
       'cs.sc', 'cs.sd', 'cs.se', 'cs.sy', 'econ.em', 'econ.gn',
       'econ.th', 'eess.as', 'eess.iv', 'eess.sp', 'eess.sy', 'math.ac',
       'math.ap', 'math.at', 'math.ca', 'math.ct', 'math.cv', 'math.dg',
       'math.ds', 'math.fa', 'math.gm', 'math.gn', 'math.gr', 'math.gt',
       'math.ho', 'math.it', 'math.kt', 'math.lo', 'math.mg', 'math.mp',
       'math.na', 'math.nt', 'math.oa', 'math.oc', 'math.qa', 'math.ra',
       'math.rt', 'math.sg', 'math.sp', 'math.st', 'nlin.ao', 'nlin.cd',
       'nlin.cg', 'nlin.ps', 'nlin.si', 'physics.acc-ph', 'physics.ao-ph',
       'physics.app-ph', 'physics.atm-clus', 'physics.bio-ph',
       'physics.chem-ph', 'physics.class-ph', 'physics.comp-ph',
       'physics.data-an', 'physics.ed-ph', 'physics.flu-dyn',
       'physics.gen-ph', 'physics.geo-ph', 'physics.hist-ph',
       'physics.ins-det', 'physics.med-ph', 'physics.optics',
       'physics.plasm-ph', 'physics.pop-ph', 'physics.soc-ph', 'q-bio.bm',
       'q-bio.cb', 'q-bio.gn', 'q-bio.mn', 'q-bio.nc', 'q-bio.ot',
       'q-bio.pe', 'q-bio.qm', 'q-bio.sc', 'q-bio.to', 'q-fin.cp',
       'q-fin.ec', 'q-fin.gn', 'q-fin.mf', 'q-fin.pm', 'q-fin.pr',
       'q-fin.rm', 'q-fin.st', 'q-fin.tr', 'stat.ap', 'stat.co',
       'stat.me', 'stat.ml', 'stat.ot', 'doi_cites_min_doi_id_label',
       'doi_cites_min_pub_publisher_label',
       'doi_cites_median_pub_publisher_label', 'doi_cites_min_update_ym',
       'doi_cites_min_first_created_ym', 'doi_cites_min_license_label',
       'doi_cites_max_license_label', 'doi_cites_q10_license_label',
       'doi_cites_q75_license_label', 'doi_cites_min_category_main_label',
       'doi_cites_q10_category_main_label',
       'doi_cites_q25_category_main_label',
       'doi_cites_min_category_main_detail_label',
       'doi_cites_median_category_main_detail_label',
       'doi_cites_q10_category_main_detail_label',
       'doi_cites_q25_category_main_detail_label',
       'doi_cites_min_category_name_parent_label',
       'doi_cites_q10_category_name_parent_label',
       'doi_cites_min_category_name_parent_main_label',
       'doi_cites_q10_category_name_parent_main_label',
       'doi_cites_min_category_name_label',
       'pred_doi_cites_min_doi_id_label',
       'pred_doi_cites_min_pub_publisher_label',
       'pred_doi_cites_median_pub_publisher_label',
       'pred_doi_cites_q75_pub_publisher_label',
       'pred_doi_cites_min_update_ym', 'pred_doi_cites_q10_update_ym',
       'pred_doi_cites_min_first_created_ym',
       'pred_doi_cites_q10_first_created_ym',
       'pred_doi_cites_mean_license_label',
       'pred_doi_cites_count_license_label',
       'pred_doi_cites_sum_license_label',
       'pred_doi_cites_min_license_label',
       'pred_doi_cites_std_license_label',
       'pred_doi_cites_q10_license_label',
       'pred_doi_cites_q25_license_label',
       'pred_doi_cites_q75_license_label',
       'pred_doi_cites_mean_category_main_label',
       'pred_doi_cites_min_category_main_label',
       'pred_doi_cites_median_category_main_label',
       'pred_doi_cites_q10_category_main_label',
       'pred_doi_cites_q25_category_main_label',
       'pred_doi_cites_mean_category_main_detail_label',
       'pred_doi_cites_sum_category_main_detail_label',
       'pred_doi_cites_min_category_main_detail_label',
       'pred_doi_cites_median_category_main_detail_label',
       'pred_doi_cites_q10_category_main_detail_label',
       'pred_doi_cites_q25_category_main_detail_label',
       'pred_doi_cites_q75_category_main_detail_label',
       'pred_doi_cites_min_category_name_parent_label',
       'pred_doi_cites_median_category_name_parent_label',
       'pred_doi_cites_q10_category_name_parent_label',
       'pred_doi_cites_q25_category_name_parent_label',
       'pred_doi_cites_min_category_name_parent_main_label',
       'pred_doi_cites_median_category_name_parent_main_label',
       'pred_doi_cites_q10_category_name_parent_main_label',
       'pred_doi_cites_q25_category_name_parent_main_label',
       'pred_doi_cites_min_category_name_label',
       'diff_rate_doi_cites_pred_doi_cites',
       'diff_rate_doi_cites_mean_submitter_label_pred_doi_cites_mean_submitter_label',
       'diff_rate_doi_cites_mean_doi_id_label_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_doi_id_label_pred_doi_cites_mean_doi_id_label',
       'diff_rate_doi_cites_mean_doi_id_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_author_first_label_pred_doi_cites_mean_author_first_label',
       'diff_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_doi_cites_mean_pub_publisher_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_update_ym_pred_doi_cites_mean_update_ym',
       'diff_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_update_ym_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_doi_cites_mean_first_created_ym_pred_doi_cites_mean_first_created_ym',
       'diff_rate_doi_cites_mean_license_label_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_license_label_pred_doi_cites_mean_license_label',
       'diff_rate_doi_cites_mean_license_label_pred_doi_cites_mean_license_label',
       'diff_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_doi_cites_mean_category_main_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_doi_cites_mean_category_name_parent_label',
       'diff_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_doi_id_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_update_ym',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_license_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_update_ym',
       'diff_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_license_label',
       'diff_rate_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_license_label',
       'diff_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_doi_cites_mean_category_name_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_doi_cites_mean_category_name_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_name_label_pred_doi_cites_mean_category_name_label',
       'diff_pred_doi_cites_mean_submitter_label_pred_doi_cites_mean_doi_id_label',
       'diff_rate_pred_doi_cites_mean_submitter_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_author_first_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_update_ym',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_first_created_ym',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_license_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_doi_id_label_pred_doi_cites_mean_category_name_label',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_pub_publisher_label',
       'diff_rate_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_update_ym',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_pred_doi_cites_mean_author_first_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_update_ym',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_first_created_ym',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_first_created_ym',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_license_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_license_label',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_pub_publisher_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_license_label',
       'diff_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_name_parent_label',
       'diff_pred_doi_cites_mean_update_ym_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_first_created_ym_pred_doi_cites_mean_license_label',
       'diff_rate_pred_doi_cites_mean_first_created_ym_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_label',
       'diff_rate_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_pred_doi_cites_mean_license_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_rate_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_main_detail_label',
       'diff_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_rate_pred_doi_cites_mean_category_main_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_label',
       'diff_rate_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_parent_main_label',
       'diff_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_category_main_detail_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_category_name_parent_label_pred_doi_cites_mean_category_name_label',
       'diff_pred_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_name_label',
       'diff_rate_pred_doi_cites_mean_category_name_parent_main_label_pred_doi_cites_mean_category_name_label',
       'is_null_comments', 'is_null_journal-ref']
importance_0_cols2.extend(['authors', 'title', 'comments',
    'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique',
    'submitter_label','doi_id_label','author_first_label','pub_publisher_label',
    'license_label','category_main_label','category_name_parent_label','category_name_parent_main_label', 'category_name_label', 'cites'
    ])

In [105]:

X = df_result.copy()
y = X['y'].values
X = X.drop(['id', 'y'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=0)
mean_squared_error(y_test, X_test.mean(axis=1), squared=False)

0.495677909118905

In [168]:
X = df_result.copy()
# train
#train = df_train.copy()
#train = train.drop(importance_0_cols2, axis=1)
#X = pd.merge(X, train, on='id', how='left')

y = X['y'].values
X = X.drop(['id', 'y'], axis=1)
X['mean'] = X.mean(axis=1)
#X = X.drop(['rf', 'lgb', 'reg', 'cat'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, np.array(y, dtype=np.float64), train_size=0.1, random_state=0)
#model = linear_model.ElasticNet(alpha=0.7, l1_ratio=0.7, max_iter=5000)
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'random_state': SEED,
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'num_iterations': 10000,
    'early_stopping_round': 50
}
model = lgb.LGBMRegressor(**lgb_params)
model.fit(X_train, y_train,
    eval_set=(X_test, y_test),
    eval_metric='rmse',
    verbose=100,
)

pred = model.predict(X_test)
mean_squared_error(y_test, pred, squared=False)

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.645232
[200]	valid_0's rmse: 0.533321
[300]	valid_0's rmse: 0.519698
Early stopping, best iteration is:
[325]	valid_0's rmse: 0.5194


0.5194004739849023

In [172]:
np.expm1(pred)

array([49.87204595, 10.43947197,  5.77703491, ..., 21.71168656,
        3.04065107, 11.57012645])

In [125]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'dart',
}
best_params, history = {}, []
trains = lgb.Dataset(X_train, y_train)
valids = lgb.Dataset(X_test, y_test)

best = lgbo.train(params, trains, valid_sets=valids,
                    verbose_eval=False,)
best.params

:15:31,769] Trial 38 finished with value: 0.5057563885089879 and parameters: {'feature_fraction': 0.48000000000000004}. Best is trial 37 with value: 0.5057563885089877.
feature_fraction_stage2, val_score: 0.505756:  67%|######6   | 2/3 [00:02<00:01,  1.25s/it][LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006654 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 1511, number of used features: 5
[LightGBM] [Info] Start training from score 2.583781
regularization_factors, val_score: 0.505756:   0%|          | 0/20 [00:00<?, ?it/s][LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031663 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 1511, number of used features: 5
[LightGBM] [Info] Start 

{'objective': 'regression',
 'metric': 'rmse',
 'boosting': 'dart',
 'feature_pre_filter': False,
 'lambda_l1': 0.00345950495765581,
 'lambda_l2': 1.0575308851308875e-08,
 'num_leaves': 2,
 'feature_fraction': 0.41600000000000004,
 'bagging_fraction': 0.8953021991684762,
 'bagging_freq': 5,
 'min_child_samples': 5,
 'num_iterations': 1000,
 'early_stopping_round': None}

In [159]:
pd.concat([X_test.reset_index(drop=True), pd.DataFrame(y_test).rename(columns={0:'y'}), pd.DataFrame(pred).rename(columns={0:'proba'})], axis=1)

,mean,y,proba
0,4.228942,4.634729,4.100452
1,2.564596,2.484907,2.440408
2,1.934328,1.609438,1.942481
3,3.224041,3.178054,3.153585
4,2.131268,2.639057,2.200531
...,...,...,...
13601,2.830482,3.583519,2.916153
13602,0.976814,1.098612,0.799299
13603,3.156468,2.944439,2.962118
13604,1.317303,2.708050,1.374299


In [161]:
X_test.dtypes

mean    float64
dtype: object

In [116]:
len(X_test), len(y_test), len(pred)

(13606, 13606, 13606)

In [114]:
pred

array([3.98169748, 2.50585187, 1.97973932, ..., 3.16836493, 1.47949482,
       2.46380513])